# LELA32052 Computational Linguistics Week 4

## N-gram language models

In this seminar we will examine n-gram language models. The first thing that we will do is to build a table of bigram counts.

In order to this I will need to introduce you to another data structure in Python - the dictionary.


### Dictionaries
In an earlier session you encountered the List. A second useful Python data structure is the Dictionary. This stores data in key and value pairs. There is a flexibility in the data types that can be keys and can be values, for example the former could be a string or an int. The latter could be a list or even another dictionary.

In [ ]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

In [ ]:
print(thisdict)

You can obtain the keys as a standalone list

In [ ]:
thisdict.keys()

And the same for the values

In [ ]:
thisdict.values()

One useful additional thing to consider is that there are different kinds of dictionaries in the Collections library. We will make use of one special kind of dictionary later - the default dictionary which returns a default value when asked for a missing key.

### For Loops

A commonly used tool in programming is the "for loop". In its simplest form this allows us to iterate over (move through) a series of values. For example:

In [ ]:
for i in range(0,10):
  print(i)

It can also be used to iterate through data structures:


In [ ]:
sentence=["the","boy","went","to","the","park"]
for word in sentence:
  print(word)

You can iterate over the keys and values of the dictionary that we created above as follows:

In [ ]:
for key, value in thisdict.items():
    print(key + " " + str(value))

### Bigram Counts

We are going to extract bigram counts from a text by using a for loop to iterate over our text, and a dictionary (specifically a defaultdict) to store the counts

First we prepare the text by tokenising it into a list of words. We also add a sentence boundary character "eol"

In [ ]:
import re
from collections import defaultdict
import numpy as np
import pandas as pd
# download from from the internet
!wget https://www.gutenberg.org/files/2554/2554-0.txt
# read in the file
f = open('2554-0.txt')
c_and_p = f.read()
# Remove the title page etc
# convert text to lower case
c_and_p=c_and_p[5464:]
c_and_p=c_and_p.lower()
c_and_p=re.sub('\n',' ', c_and_p)
# Add end of sentence token
c_and_p=re.sub("\. "," eol ", c_and_p)
c_and_p=re.sub('[^a-z ]','', c_and_p)
c_and_p=re.sub(' +', ' ',c_and_p)
c_and_p=re.split(" ", c_and_p)


In [ ]:
c_and_p

Then we iterate over the text to extract bigram counts

In [ ]:
from collections import defaultdict
total_unigrams = len(c_and_p) - 1
bigrams = defaultdict(int)
unigrams = defaultdict(int)
for i in range(total_unigrams-2):
    unigrams[c_and_p[i]] += 1
    bigrams[str.join(" ",c_and_p[i:i+2])] += 1

### Calculating the probabilities of sentences
We can then use the chain rule with a Markov assumption in order to calculate the probability of sentences:

# $p(the\ dog\ runs) = p( the| eol ) * p(dog|the) * p(runs|dog)$

In order to deal with unseen bigrams we will use add-one smoothing

In [ ]:
sentence="the man came out"
sentence=sentence.split()
sentence.insert(0,"eol")
pr=1
for i in range(len(sentence)-1):
    ugr = sentence[i]
    bgr = sentence[i] + " " + sentence[i+1]
    pr *= (bigrams[bgr]+1)/(unigrams[ugr]+len(unigrams))
format(pr, '.50f')

Problem 1a: See what the highest probability 5 word sentence you can come up with is.

Problem 1b: See what the lowest probability 5 word sentence you can come up with is.

# Generation with language models

Just as we can calculate the probability of a known string using the chain rule and Markov assumption, we can also incrementally generate sentences via a "Markov process". The probability of any sentence being generated can be calculated using chain rule.

# $p(the\ dog\ runs) = p( the| eol ) * p(dog|the) * p(runs|dog)$

![decoding](https://raw.githubusercontent.com/cbannard/compling24/refs/heads/main/images/decoding.png)

### Greedy Search

In order to incrementally generate sentences the most obvious way to proceed is just to output the most probable word at each step.

![greedy](https://raw.githubusercontent.com/cbannard/compling24/refs/heads/main/images/greedysearch.png)

### While loops
In generating sentences we are going to make use of a second very useful type of loop - the while loop. This allows us to repeatedly perform some operation while a particular statement is true. For example

In [ ]:
i=0
while i < 10:
  i=i+1
  print(i)

Before we start generating we are going to convert our bigrams counts into probabilities for convenience.

In [ ]:
nested_dict = lambda: defaultdict(nested_dict)
d = nested_dict()

for bg in bigrams:
  ug = bg.split()
  print(bg)
  d[ug[1]][ug[0]] = bigrams[bg]/unigrams[ug[0]]

lm=pd.DataFrame(d)
lm

Try greedy decoding by running the code below with different starting words. What do you notice?

In [ ]:
# Define starting word
w="i"
# Define stopping point - here when an end of line character is output
while w != "eol":
  print(w,end=' ')
  # get probabilities for all words following the previous word
  s=lm.loc[w]
  # sort the probabilities and output the most likely word
  w=s.sort_values(ascending=False).index[0]


### Sampling

Another way to pick words to output is to randomly choose them, but weighted by their probability in context. The probability then is the proportion of runs for which that word will be chosen.

![sampling](https://raw.githubusercontent.com/cbannard/compling24/refs/heads/main/images/sampling.png)

In [ ]:
import numpy as np
# Specify starting word
w="he"
# Define stopping point - here when an end of line character is output
while w != "eol":
  print(w,end=' ')
  # get probabilities for all words following the previous word
  s=lm.loc[w]
  s=s.drop(s[np.isnan(s)].index)
  # Choose randomly from the probability distribution over next words
  w=np.random.choice(list(s.index),1,list(np.exp(s.values)))[0]

What problems do you notice with these sampled sentences?

### Generating with GPT

The problems that you see with the generated output above isn't just a result of the simple bigram model used to generate the probabilities. The same problems apply even when using a more sophisticated language model such as GPT.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

### Greedy Search Generation with GPT-2

In [ ]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

### Pure sampling Generation with GPT-2

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

# Top-k sampling

As a response to the problems seen with these sampling methods, researchers have come up with different methods. An example is top-K sampling, which was used in the released GPT-2 model. In this method we sample, but we do so from the top-K words rather than all the vocabulary. This excludes the less likely words that meant the texts were not meaningful.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)


In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))